Run the below cell to load training images, it will take about 2 minutes for the dataset used in the thesis. Then run the second cell to train a CNN model. The third cell can be run without the first and second cell run, and is used to re-evaluate any previously trained model on the test dataset.

You will need the following folders and files in the working directory for this to work:
- test_timestacks/
- training_timestack_snapshots/
- model folder (if testing a previously trained model)

Make sure you rename the tf_model_version when you want to train a different model, if the same name is used it will overwrite the previous model trained.

In [ ]:
# import libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import listdir
import random
import csv

# set the image size you want as input to the model
IMG_SIZE = 64
IMG_SIZE_WID = 64
training_folder = "training_timestack_snapshots"

# function to evaluate a model on the test timestacks
def compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, 
                      IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, exe_value):
    
    folder_name = "test_timestacks/"
    
    uprush_width = max_uprush - min_uprush
    
    # add % to min and max uprush
    min_uprush = int(min_uprush - uprush_width*(exc_value/200))
    max_uprush = int(max_uprush + uprush_width*(exc_value/200))
    
    # add % to height of two peaks
    height_of_two_peaks = int(height_of_two_peaks*(1 + eve_value/100))
    
    timestack = cv2.imread(folder_name + timestack_name)
    timestack_focus = timestack[:, min_uprush:max_uprush]
    timestack_labelled = cv2.imread(folder_name + timestack_name + "_output.png")
    timestack_annotated = timestack_labelled.copy()
    
    padding = height_of_two_peaks
    height, width, _ = timestack.shape
    width = max_uprush - min_uprush
    timestack_count = 0

    tf_images = []
    tf_model_xc = tf.keras.models.load_model(tf_model_version + ".model")

    vertical_coordinates = []
    horizontal_coordinates = []
    tf_images_list = []
    for i in range(0, int((height - padding)), 1):

        # get the small section of the image for the horizontal position i
        timestack_snapshot = timestack_focus[i:i + padding, :]
        
        # classify the timestack snapshot through the convolutional neural network for x coordinate
        tf_images = cv2.resize(timestack_snapshot, (IMG_SIZE_WID, IMG_SIZE))
        tf_images_list.append(tf_images)
    
    tf_images_list = np.array(tf_images_list)
    tf_images_list = tf.keras.utils.normalize(tf_images_list, axis=1)
    tf_images_list = tf_images_list.reshape(-1, IMG_SIZE_WID, IMG_SIZE, 3)
    
    predictions = tf_model_xc.predict([tf_images_list])
    
    for i in range(0, int((height - padding)), 1):
        
        horizontal_coordinate = i + int(padding/2)      
        vertical_coordinate = int((predictions[i][0])*width) + min_uprush
        #vertical_coordinate = int(min_uprush + width/2) # use this line for straight baseline model calculation
        
        cv2.circle(timestack_annotated, (vertical_coordinate, horizontal_coordinate), radius=1, color=[255, 0, 0])
        vertical_coordinates.append(vertical_coordinate)
        horizontal_coordinates.append(horizontal_coordinate)
        timestack_count = timestack_count + 1
            
    # get csv values from labelled timestack
    manual_csv_points_filename = "runup_data_test" + timestack_name + ".csv"
    x_vals = []
    t_vals = []
    minima_test = []
    maxima_test = []
    with open(folder_name + manual_csv_points_filename, newline='') as csvfile:
        points_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        initial_row = 5
        initial_row_counter = 0
        for row in points_reader:
            if initial_row_counter > initial_row:
                rows = row[0].split(',')
                x_vals.append( int(rows[0][1:-1]) )
                t_vals.append( int(rows[1][1:-1]) )
                # also extract min and max values
                minima_test.append( int(rows[2][1:-1]) )
                maxima_test.append( int(rows[3][1:-1]) )
                
            initial_row_counter = initial_row_counter + 1

    full_stack_window = "timestack_annotated"

    # find MSE for evey row
    SE = []
    minima_maxima_test_coords = []
    for i in range(0, len(t_vals), 1):
        # make sure the comparison values line up correctly
        index_matching = np.where(np.array(horizontal_coordinates) == t_vals[i])
        
        # ensure prediction and manual points are both present
        if len(index_matching[0]) < 1:
            continue
        
        vertical_coordinate_matching = vertical_coordinates[ index_matching[0][0] ]
        SE.append( (100*(x_vals[i] - vertical_coordinate_matching)/(uprush_width))**2 )
        
        # find minima and maxima values for predicted values
        if minima_test[i] == 1:
            minima_maxima_test_coords.append([x_vals[i], t_vals[i], -1])
        elif maxima_test[i] == 1:
            minima_maxima_test_coords.append([x_vals[i], t_vals[i], 1])
    
    RMSE = (np.mean(SE))**0.5
    
    SE_min = []
    SE_max = []
    # for each minima or maxima values excluding first and last value
    for i in range(1, len(minima_maxima_test_coords)-1, 1):
        t_limit_1 = minima_maxima_test_coords[i-1][1] + (minima_maxima_test_coords[i][1] - minima_maxima_test_coords[i-1][1])/2
        t_limit_2 = minima_maxima_test_coords[i][1] + (minima_maxima_test_coords[i+1][1] - minima_maxima_test_coords[i][1])/2
        
        # between the two time limits, find the minimum predicted value
        if minima_maxima_test_coords[i][2] == -1:
            
            indxs = np.where(np.logical_and(np.array(horizontal_coordinates) > t_limit_1, 
                                            np.array(horizontal_coordinates) < t_limit_2))
            pred_min_x = np.min(np.array(vertical_coordinates)[indxs])
            pred_min_t_ind = np.argmin(np.array(vertical_coordinates)[indxs])
            pred_min_t = horizontal_coordinates[indxs[0][pred_min_t_ind]]

            # plot on annotated image
            cv2.circle(timestack_annotated, (minima_maxima_test_coords[i][0], minima_maxima_test_coords[i][1]), 
                       radius=1, color=[0, 255, 150])
            cv2.circle(timestack_annotated, (pred_min_x, pred_min_t), radius=1, color=[255, 255, 0])
            
            SE_min.append( (100*(minima_maxima_test_coords[i][0] - pred_min_x)/(uprush_width))**2 )
            
        # between the two time limits, find the maximum predicted value
        elif minima_maxima_test_coords[i][2] == 1:
            
            indxs = np.where(np.logical_and(np.array(horizontal_coordinates) > t_limit_1, 
                                            np.array(horizontal_coordinates) < t_limit_2))
            pred_max_x = np.max(np.array(vertical_coordinates)[indxs])
            pred_max_t_ind = np.argmax(np.array(vertical_coordinates)[indxs])
            pred_max_t = horizontal_coordinates[indxs[0][pred_max_t_ind]]
            
            # plot on annotated image
            cv2.circle(timestack_annotated, (minima_maxima_test_coords[i][0], minima_maxima_test_coords[i][1]), 
                       radius=1, color=[0, 255, 150])
            cv2.circle(timestack_annotated, (pred_max_x, pred_max_t), radius=1, color=[255, 255, 0])
            
            SE_max.append( (100*(minima_maxima_test_coords[i][0] - pred_max_x)/(uprush_width))**2 )
    
    # get RMSE between min pairs and max pairs normalised by the range
    RMSE_min = (np.mean(SE_min))**0.5
    RMSE_max = (np.mean(SE_max))**0.5
    
    # save the image
    print("saving image")
    cv2.imwrite("annotated_runup_timestack_" + timestack_name + "_" + tf_model_version + ".png", timestack_annotated)
    
    # save the data
    print("Saving Data to csv file")
    # create csv file
    output_data_name = "runup_data_" + timestack_name + "_" + tf_model_version + ".csv"
    with open(output_data_name, 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows([["timestack shoreline coordinates"]])

    with open(output_data_name, 'a', newline='') as csvFile:
        writer = csv.writer(csvFile)
        time_row = list(["time (horizontal coordinate)"])
        time_row.extend(horizontal_coordinates)
        x_row = list(["x (vertical coordinate)"])
        x_row.extend(vertical_coordinates)
        writer.writerow(time_row)
        writer.writerow(x_row)

    csvFile.close()
    
    return RMSE, RMSE_min, RMSE_max

"""%%%%%%%%%%%%%%%%%%%% GET IMAGES %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
data = []
training_test_ratio = 0.85
exc_value = 0 # % excursion range increase or decrease
eve_value = 0 # % events increase or decrease

# first get all images and label accordingly
image_names = listdir(training_folder)
for i in range(0, len(image_names), 1):
    if i % 1000 == 0:
        print(i)
    
    image_i = cv2.imread(training_folder + "/" + image_names[i])
    height, _, _ = image_i.shape
    # below line takes 3/10ths from each side of 5 peaks to give 2 peaks.
    # note that /4 was used for the final model analysed in the thesis
    # but should have been /3.3. A model was run with /3.3 and the 
    # difference in performance was negligable.
    image_i = image_i[int(height/3.3):int(height - height/3.3), :]
    resized_image = cv2.resize(image_i, (IMG_SIZE_WID, IMG_SIZE))
    resized_image_flipped = cv2.flip(resized_image, 0)
    Xc = float(image_names[i][23:28])
    data.append([resized_image, Xc])
    data.append([resized_image_flipped, Xc])
    
# Randomize rows of data
random.shuffle(data)

# Setup a training set and test set
data_train = data[0:int(training_test_ratio*len(data))]
data_test = data[int(training_test_ratio*len(data)):]

# Split x data from y data
x_train = np.array(list(map(list, zip(*data_train)))[0])
y_train = np.array(list(map(list, zip(*data_train)))[1])

# setup data for tensorflow
x_test = np.array(list(map(list, zip(*data_test)))[0])
y_test = np.array(list(map(list, zip(*data_test)))[1])

# normalize input features (which are the pixel values)
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

# reshape for convolutional compatability
x_train = x_train.reshape(-1, IMG_SIZE_WID, IMG_SIZE, 3)
x_test = x_test.reshape(-1, IMG_SIZE_WID, IMG_SIZE, 3)

print("done")

In [ ]:
"""%%%%%%%%%%%%%%%%%%%% MODEL SETUP AND TRAINING %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
epochs_no = 50
batch_size_no = 512
tf_model_version = "_v1"

model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = x_train.shape[1:]))
model.add(Activation("relu"))
model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2,2), padding="SAME"))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2,2), padding="SAME"))

model.add(Conv2D(128, (3,3)))
model.add(Activation("relu"))
model.add(Conv2D(128, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2,2), padding="SAME"))

model.add(Conv2D(256, (3,3)))
model.add(Activation("relu"))
model.add(Conv2D(256, (3,3)))
model.add(Activation("relu"))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Dense(512))

model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss="mean_squared_error", optimizer="adam", metrics=['mean_squared_error'])
history = model.fit(x_train, y_train, batch_size=batch_size_no, validation_split=0.1, epochs=epochs_no)

"""%%%%%%%%%%%%%%%%%%%% MODEL EVALUATION %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
train_loss, train_acc = model.evaluate(x_train, y_train)
val_loss, val_acc = model.evaluate(x_test, y_test)

print("training loss = " + str(train_loss))
print("val loss     = " + str(val_loss))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

model.save("Timestack_Shoreline_Xc" + tf_model_version + ".model")

error_print_list = []

timestack_name = "20101109084810_02_25ppm_test_s.png"
min_uprush = 296
max_uprush = 744
height_of_two_peaks = int(209*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       "Timestack_Shoreline_Xc" + tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "20101110094203_05_25ppm_test_s.png"
min_uprush = 93
max_uprush = 546
height_of_two_peaks = int(280*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       "Timestack_Shoreline_Xc" + tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "20101109103953_08_25ppm_test_s.png"
min_uprush = 194
max_uprush = 687
height_of_two_peaks = int(238*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       "Timestack_Shoreline_Xc" + tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "20101111092821_01_25ppm_test_s.png"
min_uprush = 141
max_uprush = 779
height_of_two_peaks = int(353*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       "Timestack_Shoreline_Xc" + tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "moreton_12_test_s.png"
min_uprush = 117
max_uprush = 235
height_of_two_peaks = int(300*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       "Timestack_Shoreline_Xc" + tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "OneMile8_test_s.png"
min_uprush = 105
max_uprush = 251
height_of_two_peaks = int(258*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       "Timestack_Shoreline_Xc" + tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "FC4_test_s.png"
min_uprush = 51
max_uprush = 205
height_of_two_peaks = int(400*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       "Timestack_Shoreline_Xc" + tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "Werri31_test_s.png"
min_uprush = 71
max_uprush = 203
height_of_two_peaks = int(213*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       "Timestack_Shoreline_Xc" + tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

for err_val in error_print_list:
    print(err_val)

print("done")

This cell below can be used to re-evaluate any model without having to retrain a new model. You can also change the excursion and events values to perform some sensitivity analysis for user defined parameters.

In [ ]:
tf_model_version = "Timestack_Shoreline_Xc" + "_v0" # v0 is the model from the thesis
IMG_SIZE = 64
IMG_SIZE_WID = 64
exc_value = 0 # % excursion range increase or decrease
eve_value = 0 # % events increase or decrease

# import libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import listdir
import random
import csv

# function to evaluate a model on the test timestacks
def compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, 
                      IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, exe_value):
    
    folder_name = "test_timestacks/"
    
    uprush_width = max_uprush - min_uprush
    
    # add % to min and max uprush
    min_uprush = int(min_uprush - uprush_width*(exc_value/200))
    max_uprush = int(max_uprush + uprush_width*(exc_value/200))
    
    # add % to height of two peaks
    height_of_two_peaks = int(height_of_two_peaks*(1 + eve_value/100))
    
    timestack = cv2.imread(folder_name + timestack_name)
    timestack_focus = timestack[:, min_uprush:max_uprush]
    timestack_labelled = cv2.imread(folder_name + timestack_name + "_output.png")
    timestack_annotated = timestack_labelled.copy()
    
    padding = height_of_two_peaks
    height, width, _ = timestack.shape
    width = max_uprush - min_uprush
    timestack_count = 0

    tf_images = []
    tf_model_xc = tf.keras.models.load_model(tf_model_version + ".model")

    vertical_coordinates = []
    horizontal_coordinates = []
    tf_images_list = []
    for i in range(0, int((height - padding)), 1):

        # get the small section of the image for the horizontal position i
        timestack_snapshot = timestack_focus[i:i + padding, :]
        
        # classify the timestack snapshot through the convolutional neural network for x coordinate
        tf_images = cv2.resize(timestack_snapshot, (IMG_SIZE_WID, IMG_SIZE))
        tf_images_list.append(tf_images)
    
    tf_images_list = np.array(tf_images_list)
    tf_images_list = tf.keras.utils.normalize(tf_images_list, axis=1)
    tf_images_list = tf_images_list.reshape(-1, IMG_SIZE_WID, IMG_SIZE, 3)
    
    predictions = tf_model_xc.predict([tf_images_list])
    
    for i in range(0, int((height - padding)), 1):
        
        horizontal_coordinate = i + int(padding/2)      
        vertical_coordinate = int((predictions[i][0])*width) + min_uprush
        #vertical_coordinate = int(min_uprush + width/2) # use this line for straight baseline model calculation
        
        cv2.circle(timestack_annotated, (vertical_coordinate, horizontal_coordinate), radius=1, color=[255, 0, 0])
        vertical_coordinates.append(vertical_coordinate)
        horizontal_coordinates.append(horizontal_coordinate)
        timestack_count = timestack_count + 1
            
    # get csv values from labelled timestack
    manual_csv_points_filename = "runup_data_test" + timestack_name + ".csv"
    x_vals = []
    t_vals = []
    minima_test = []
    maxima_test = []
    with open(folder_name + manual_csv_points_filename, newline='') as csvfile:
        points_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        initial_row = 5
        initial_row_counter = 0
        for row in points_reader:
            if initial_row_counter > initial_row:
                rows = row[0].split(',')
                x_vals.append( int(rows[0][1:-1]) )
                t_vals.append( int(rows[1][1:-1]) )
                # also extract min and max values
                minima_test.append( int(rows[2][1:-1]) )
                maxima_test.append( int(rows[3][1:-1]) )
                
            initial_row_counter = initial_row_counter + 1

    full_stack_window = "timestack_annotated"

    # find MSE for evey row
    SE = []
    minima_maxima_test_coords = []
    for i in range(0, len(t_vals), 1):
        # make sure the comparison values line up correctly
        index_matching = np.where(np.array(horizontal_coordinates) == t_vals[i])
        
        # ensure prediction and manual points are both present
        if len(index_matching[0]) < 1:
            continue
        
        vertical_coordinate_matching = vertical_coordinates[ index_matching[0][0] ]
        SE.append( (100*(x_vals[i] - vertical_coordinate_matching)/(uprush_width))**2 )
        
        # find minima and maxima values for predicted values
        if minima_test[i] == 1:
            minima_maxima_test_coords.append([x_vals[i], t_vals[i], -1])
        elif maxima_test[i] == 1:
            minima_maxima_test_coords.append([x_vals[i], t_vals[i], 1])
    
    RMSE = (np.mean(SE))**0.5
    
    SE_min = []
    SE_max = []
    # for each minima or maxima values excluding first and last value
    for i in range(1, len(minima_maxima_test_coords)-1, 1):
        t_limit_1 = minima_maxima_test_coords[i-1][1] + (minima_maxima_test_coords[i][1] - minima_maxima_test_coords[i-1][1])/2
        t_limit_2 = minima_maxima_test_coords[i][1] + (minima_maxima_test_coords[i+1][1] - minima_maxima_test_coords[i][1])/2
        
        # between the two time limits, find the minimum predicted value
        if minima_maxima_test_coords[i][2] == -1:
            
            indxs = np.where(np.logical_and(np.array(horizontal_coordinates) > t_limit_1, 
                                            np.array(horizontal_coordinates) < t_limit_2))
            pred_min_x = np.min(np.array(vertical_coordinates)[indxs])
            pred_min_t_ind = np.argmin(np.array(vertical_coordinates)[indxs])
            pred_min_t = horizontal_coordinates[indxs[0][pred_min_t_ind]]

            # plot on annotated image
            cv2.circle(timestack_annotated, (minima_maxima_test_coords[i][0], minima_maxima_test_coords[i][1]), 
                       radius=1, color=[0, 255, 150])
            cv2.circle(timestack_annotated, (pred_min_x, pred_min_t), radius=1, color=[255, 255, 0])
            
            SE_min.append( (100*(minima_maxima_test_coords[i][0] - pred_min_x)/(uprush_width))**2 )
            
        # between the two time limits, find the maximum predicted value
        elif minima_maxima_test_coords[i][2] == 1:
            
            indxs = np.where(np.logical_and(np.array(horizontal_coordinates) > t_limit_1, 
                                            np.array(horizontal_coordinates) < t_limit_2))
            pred_max_x = np.max(np.array(vertical_coordinates)[indxs])
            pred_max_t_ind = np.argmax(np.array(vertical_coordinates)[indxs])
            pred_max_t = horizontal_coordinates[indxs[0][pred_max_t_ind]]
            
            # plot on annotated image
            cv2.circle(timestack_annotated, (minima_maxima_test_coords[i][0], minima_maxima_test_coords[i][1]), 
                       radius=1, color=[0, 255, 150])
            cv2.circle(timestack_annotated, (pred_max_x, pred_max_t), radius=1, color=[255, 255, 0])
            
            SE_max.append( (100*(minima_maxima_test_coords[i][0] - pred_max_x)/(uprush_width))**2 )
    
    # get RMSE between min pairs and max pairs normalised by the range
    RMSE_min = (np.mean(SE_min))**0.5
    RMSE_max = (np.mean(SE_max))**0.5
    
    # save the image
    print("saving image")
    cv2.imwrite("annotated_runup_timestack_" + timestack_name + "_" + tf_model_version + ".png", timestack_annotated)
    
    # save the data
    print("Saving Data to csv file")
    # create csv file
    output_data_name = "runup_data_" + timestack_name + "_" + tf_model_version + ".csv"
    with open(output_data_name, 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows([["timestack shoreline coordinates"]])

    with open(output_data_name, 'a', newline='') as csvFile:
        writer = csv.writer(csvFile)
        time_row = list(["time (horizontal coordinate)"])
        time_row.extend(horizontal_coordinates)
        x_row = list(["x (vertical coordinate)"])
        x_row.extend(vertical_coordinates)
        writer.writerow(time_row)
        writer.writerow(x_row)

    csvFile.close()
    
    return RMSE, RMSE_min, RMSE_max

new_model = tf.keras.models.load_model(tf_model_version + ".model")
new_model.summary()

error_print_list = []

timestack_name = "20101109084810_02_25ppm_test_s.png"
min_uprush = 296
max_uprush = 744
height_of_two_peaks = int(209*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "20101110094203_05_25ppm_test_s.png"
min_uprush = 93
max_uprush = 546
height_of_two_peaks = int(280*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "20101109103953_08_25ppm_test_s.png"
min_uprush = 194
max_uprush = 687
height_of_two_peaks = int(238*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "20101111092821_01_25ppm_test_s.png"
min_uprush = 141
max_uprush = 779
height_of_two_peaks = int(353*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + " is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "moreton_12_test_s.png"
min_uprush = 117
max_uprush = 235
height_of_two_peaks = int(300*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + "              is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "OneMile8_test_s.png"
min_uprush = 105
max_uprush = 251
height_of_two_peaks = int(258*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + "                is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "FC4_test_s.png"
min_uprush = 51
max_uprush = 205
height_of_two_peaks = int(400*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + "                     is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

timestack_name = "Werri31_test_s.png"
min_uprush = 71
max_uprush = 203
height_of_two_peaks = int(213*2/5)
RMSE, RMSE_min, RMSE_max = compare_shoreline(min_uprush, max_uprush, 
                                       tf_model_version, 
                                       IMG_SIZE, IMG_SIZE_WID, timestack_name, height_of_two_peaks, exc_value, eve_value)

error_print_list.append("RMSE for " + timestack_name + "                 is " + str(round(RMSE, 2)) + "%, minima RMSE = " + 
                        str(round(RMSE_min, 2)) + "%, maxima RMSE = " + str(round(RMSE_max, 2)) + "%")

for err_val in error_print_list:
    print(err_val)

print("done")